# Pre-requirements



In [3]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c78e9768bd23a3bf61951f894be007120bd23d9eea6f7b8d4abfe064c38bf066
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


# Libraries

In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
from pyspark.sql.window import Window


# Code

In [5]:
root_dir = os.path.dirname(os.path.abspath(os.getcwd()))
test_file = os.path.join(root_dir,'content', 'coronavirus_2023.csv')
test_file

'/content/coronavirus_2023.csv'

In [6]:
# Create spark session
spark = SparkSession.builder \
    .appName("Data Quality Check") \
    .getOrCreate()

In [8]:
# Read DF To perform Data Quality Check
df = spark.read.csv(test_file, header=True, inferSchema=True)
df.show(1)

+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+
|      date|province|country|    lat|     long|     type|cases|  uid|iso2|iso3|code3|   combined_key|population|continent_name|continent_code|
+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+
|2023-01-01| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146| North America|            NA|
+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+
only showing top 1 row



## Check date format

In [9]:
# Check if date have correct format
def check_date_format(df, column_to_check, date_format):

  # Convert date column to string
  formatted_date_column = f.date_format(df[column_to_check], date_format).cast(StringType())

  # Conditional check if format of column is the same as the one specified in date_format
  is_same_format_condition = f.when(formatted_date_column != df[column_to_check], df[column_to_check])

  # Filter df with correct/incorrect formats
  correct_format_df = df.filter(is_same_format_condition.isNull())
  incorrect_format_df = df.filter(is_same_format_condition.isNotNull())

    # Change return accordingly
  return correct_format_df, incorrect_format_df

In [10]:
correct_format_df, incorrect_format_df = check_date_format(df, 'date', 'yyyy-MM-dd')
correct_format_df.show()
incorrect_format_df.show()

+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+
|      date|province|country|    lat|     long|     type|cases|  uid|iso2|iso3|code3|   combined_key|population|continent_name|continent_code|
+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+
|2023-01-01| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146| North America|            NA|
|2023-01-02| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146| North America|            NA|
|2023-01-03| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146| North America|            NA|
|2023-01-04| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146| North America|            NA|

## Check No duplicates

In [11]:
# Check if a column doesn't have duplicates
def check_no_duplicates(df, column_to_check):

    # Create a Window partition for all the unique values of the column to check
    window_partition = Window().partitionBy(column_to_check)

    # Add column with frequency per each unique value
    df_with_frequency = df.withColumn('Frequency', f.count(column_to_check).over(window_partition))

    # Filter rows where the count is greater than 1 (indicating duplicates)
    df_wth_duplicates = df_with_frequency.filter(f.col('Frequency') > 1)
    df_no_duplicates = df_with_frequency.filter(f.col('Frequency') == 1)
    # Change return accordingly
    return df_wth_duplicates, df_no_duplicates


In [12]:
df_wth_duplicates, df_no_duplicates = check_no_duplicates(df, 'province')
df_wth_duplicates.show()
df_no_duplicates.show()

+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+---------+
|      date|province|country|    lat|     long|     type|cases|  uid|iso2|iso3|code3|   combined_key|population|continent_name|continent_code|Frequency|
+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+---------+
|2023-01-01| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146| North America|            NA|      136|
|2023-01-02| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146| North America|            NA|      136|
|2023-01-03| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146| North America|            NA|      136|
|2023-01-04| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  1

## Check if Null values

In [13]:
def check_null_values(df, column_to_check):
    # Filter df with correct/incorrect formats
  df_with_null = df.filter(df[column_to_check].isNull())
  df_without_null = df.filter(df[column_to_check].isNotNull())
  return df_with_null, df_without_null


In [14]:
df_wth_duplicates = df_wth_duplicates.withColumn('continent_name', f.when(f.col('continent_name') == 'North America', None).otherwise(f.col('continent_name')))
df_wth_duplicates.show(1)
df_with_null, df_without_null = check_null_values(df_wth_duplicates, 'continent_name')
df_with_null.show(1)
df_without_null.show(1)

+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+---------+
|      date|province|country|    lat|     long|     type|cases|  uid|iso2|iso3|code3|   combined_key|population|continent_name|continent_code|Frequency|
+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+---------+
|2023-01-01| Alberta| Canada|53.9333|-116.5765|confirmed|    0|12401|  CA| CAN|  124|Alberta, Canada|   4413146|          NULL|            NA|      136|
+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+---------+
only showing top 1 row

+----------+--------+-------+-------+---------+---------+-----+-----+----+----+-----+---------------+----------+--------------+--------------+---------+
|      date|province|country|    lat|     long|     type|c

## Check if columns have duplicates

In [37]:
from pyspark.sql.functions import count, col
from pyspark.sql.window import Window

def check_no_duplicates(df, column):

    # Create a Window partition for all the unique values of the columns
    window_partition = Window().partitionBy(column)

    # Add Frequency column to check if there are duplicated rows
    df_with_frequency = df.withColumn('Frequency', count('*').over(window_partition))

    # Check if df have duplicated values in the column
    num_of_rows = df_with_frequency.filter(col('Frequency') > 1).count()
    if num_of_rows == 0:
      print(f'no duplicated values in column: {column}')
    if num_of_rows > 0:
      print(f' {num_of_rows} duplicated rows in column: {column}. Check below the duplicated row')
      df_with_frequency.filter(col('Frequency') > 1).drop('Frequency').show(1)
      # return df_with_frequency.filter(col('Frequency') > 1).drop('Frequency')



In [38]:
from pyspark.sql import SparkSession
# Initialize Spark session
spark = SparkSession.builder.appName("CheckNoDuplicatesTest").getOrCreate()

# Create example DataFrame

data = [(1, "John", 30), (2, "Alice", 25), (3, "Bob", 35), (4, "Dave", 30), (4, "Dave", 30)]
columns = ["id", "name", "age"]
df = spark.createDataFrame(data, columns)

# Specify columns to check for duplicates
specified_columns = 'id'

# Check for duplicates in specified columns
check_no_duplicates(df, specified_columns)

spark.stop()

 2 duplicated rows in column: id. Check below the duplicated row
+---+----+---+
| id|name|age|
+---+----+---+
|  4|Dave| 30|
+---+----+---+
only showing top 1 row



## Check if column values are within range


In [77]:
def check_column_range(df, column, min_value, max_value):
    # Check if df have values outside the range
    num_of_rows = df.filter((col(column) < min_value) | (col(column) > max_value)).count()
    if num_of_rows == 0:
      print(f'dataframe does not have values outside the range: {min_value}-{max_value} for the column: {column}')
    else:
      print(f'dataframe does have values outside the range: {min_value}-{max_value} for the column: {column}. Check rows below')
      df.filter((col(column) < min_value) | (col(column) > max_value)).show()



In [78]:
spark = SparkSession.builder.appName("CheckColumnRangeTest").getOrCreate()

# Create example DataFrame
data = [(1, "John", 30), (2, "Alice", 25), (3, "Bob", 35), (4, "Dave", 40)]
columns = ["id", "name", "age"]
df = spark.createDataFrame(data, columns)

# Specify the column and range to check
column_to_check = "age"
min_age = 20
max_age = 35

# Run the check
check_column_range(df, column_to_check, min_age, max_age)
spark.stop()

dataframe does have values outside the range: 20-35 for the column: age. Check rows below
+---+----+---+
| id|name|age|
+---+----+---+
|  4|Dave| 40|
+---+----+---+



## Check if string column has lenght within range

In [79]:
from pyspark.sql.functions import length

def check_string_length_range(df, column, min_length, max_length):

  # Check if any row with string lenght
  num_of_rows = df.filter((length(col(column)) < min_length) | (length(col(column)) > max_length)).count()
  if num_of_rows == 0:
      print(f'dataframe does not have values outside the range: {min_length}-{max_length} for the column: {column}')
  else:
    print(f'dataframe does have values outside the range: {min_length}-{max_length} for the column: {column}. Check rows below')
    df.filter((length(col(column)) < min_length) | (length(col(column)) > max_length)).show()


In [82]:
spark = SparkSession.builder.appName("CheckStringLengthRangeTest").getOrCreate()

# Create example DataFrame
data = [(1, "John", 30), (2, "Aliceee", 25), (3, "Bob", 35), (4, "Dave", 40)]
columns = ["id", "name", "age"]
df = spark.createDataFrame(data, columns)

# Specify the string column and length range to check
column_to_check = "name"
min_length = 3
max_length = 5

# Run the check
check_string_length_range(df, column_to_check, min_length, max_length)


# Stop the Spark session
spark.stop()

dataframe does have values outside the range: 3-5 for the column: name. Check rows below
+---+-------+---+
| id|   name|age|
+---+-------+---+
|  2|Aliceee| 25|
+---+-------+---+



## Check if text matches regex pattern

In [90]:

from pyspark.sql.functions import regexp_extract

def check_column_match_regex(df, column, regex_pattern):
  # Check if any row with string lenght
  num_of_rows = df.filter(regexp_extract(col(column), regex_pattern, 0) != "").count()
  if num_of_rows == 0:
      print(f'dataframe does not have values with the pattern: {regex_pattern} for the column: {column}')
  else:
    print(f'dataframe do have values with the pattern: {regex_pattern} for the column: {column}. Check rows below')
    df.filter(regexp_extract(col(column), regex_pattern, 0) != "").show()


In [91]:
spark = SparkSession.builder.appName("CheckColumnRegexTest").getOrCreate()

# Create example DataFrame
data = [(1, "John123", 30), (2, "Alice456", 25), (3, "Bob", 35), (4, "Dave", 40)]
columns = ["id", "name", "age"]
df = spark.createDataFrame(data, columns)

# Specify the column and regex pattern to check
column_to_check = "name"
regex_pattern = r'\d+'  # Matches one or more digits

# Run the check
check_column_match_regex(df, column_to_check, regex_pattern)
# Stop the Spark session
spark.stop()

dataframe do have values with the pattern: \d+ for the column: name. Check rows below
+---+--------+---+
| id|    name|age|
+---+--------+---+
|  1| John123| 30|
|  2|Alice456| 25|
+---+--------+---+



## Check if string column values are within a list of values

In [94]:
def check_column_values_in_list(df, column, value_list):
  # Check if any row with string lenght
  num_of_rows = df.filter(~col(column).isin(value_list)).count()
  if num_of_rows == 0:
      print(f'dataframe does not have values within the list of values: {value_list} for the column: {column}')
  else:
    print(f'dataframe do have values within the list of values: {value_list} for the column: {column}. Check rows below')
    df.filter(~col(column).isin(value_list)).show()


In [95]:
spark = SparkSession.builder.appName("CheckColumnValuesInListTest").getOrCreate()

# Create example DataFrame
data = [(1, "John", 30), (2, "Alice", 25), (3, "Bob", 35), (4, "Dave", 40)]
columns = ["id", "name", "age"]
df = spark.createDataFrame(data, columns)

# Specify the string column and list of values to check
column_to_check = "name"
values_to_check = ["John", "Alice", "Charlie"]

# Run the check
check_column_values_in_list(df, column_to_check, values_to_check)


# Stop the Spark session
spark.stop()

dataframe do have values within the list of values: ['John', 'Alice', 'Charlie'] for the column: name. Check rows below
+---+----+---+
| id|name|age|
+---+----+---+
|  3| Bob| 35|
|  4|Dave| 40|
+---+----+---+



## check if numeric column values are within a list of values (intended for categorical values)

In [ ]:
# Same function as above

## Check Number of elements per category

In [15]:
df.groupBy("").count().show()

AnalysisException: ignored

In [ ]:
df.dropDuplicates(subset = ['continent_name']).show()

In [ ]:
df2 = df.withColumn("lat",f.col('lat').cast('int'))
quantiles = df2.approxQuantile("lat", [0.01, 0.99], 0.0)
lower_bound = quantiles[0]
upper_bound = quantiles[1]

outliers = df2.filter((df2["lat"] < lower_bound) | (df2["lat"] > upper_bound))
outliers.show(100)

##Compare Schemas

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

def compare_schemas(spark, source_df, target_df):
    """
    Compare the schemas of source and target DataFrames.

    Parameters:
    - spark: PySpark SparkSession
    - source_df: PySpark DataFrame representing the source table
    - target_df: PySpark DataFrame representing the target table

    Returns:
    - True if the schemas match, False otherwise
    """
    source_schema = source_df.schema
    target_schema = target_df.schema

    if source_schema == target_schema:
        print("Schemas match. Source and target tables have the same structure.")
        return True
    else:
        print("Schemas do not match. Source and target tables have different structures.")

        source_columns = {field.name: field.dataType for field in source_schema.fields}
        target_columns = {field.name: field.dataType for field in target_schema.fields}

        different_columns = set(source_columns.keys()) ^ set(target_columns.keys())

        for column in different_columns:
            source_data_type = source_columns.get(column, "Not present in source")
            target_data_type = target_columns.get(column, "Not present in target")

            print(f"Column: {column}, Source Data Type: {source_data_type}, Target Data Type: {target_data_type}")

        return False
# Create a Spark session
spark = SparkSession.builder.appName("SchemaComparisonTest").getOrCreate()

# Define schemas for source and target DataFrames
source_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

target_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("score", IntegerType(), True)
])

# Sample data for source and target DataFrames
source_data = [(1, "Alice", 25), (2, "Bob", 30), (3, "Charlie", 22)]
target_data = [(1, "Alice", 95), (2, "Bob", 80), (3, "Charlie", 75)]

# Create source and target DataFrames
source_df = spark.createDataFrame(source_data, schema=source_schema)
target_df = spark.createDataFrame(target_data, schema=target_schema)

# Print the content of source and target DataFrames
print("Source DataFrame:")
source_df.show()

print("Target DataFrame:")
target_df.show()
# Compare Schemas
compare_schemas(spark, source_df, target_df)
# Stop the Spark session
spark.stop()


# Create framework for checker

There are 3 types of validations. :

- Entity
- Field
- Data


- Check that source column have the same range values as target colum
- Check that source column have the same len than target column
- Check Column names are same as desired (config file)
- verify all field required from source talbe are in target table
- Capture constraints defined in sourced table (what constraints do we have? E.g. maybe another config file with a function to find if a number is negative)
-